In [1]:
#This cell is used to import modules that will be useful for model prediction. 

import pandas as pd #Pandas is the base Python library used to manuipulate datasets, typically this will always be useful. 
import numpy as np #Numpy is mainly used to allow for mathematical manipulations of datasets, typically this will always be useful.



In [2]:
#Importing datasets
df_original = pd.read_csv('Datasets/living cost.csv') #Used to read .csv files.
#df2 = pd.read_excel('Datasets/Global Superstore.xls', sheet_name='People') #Used to read .xls files, sheet_name can be used to select for different sheets by using index number or string name.
df_original.head() #Initial dataset without any manipulation.

,Unnamed: 0,City,Cost_index,"Meal, Inexpensive Restaurant","Meal for 2 People, Mid-range Restaurant, Three-course",McMeal at McDonalds (or Equivalent Combo Meal),Domestic Beer (0.5 liter draught),Imported Beer (0.33 liter bottle),Cappuccino (regular),Coke/Pepsi (0.33 liter bottle),...,"1 Summer Dress in a Chain Store (Zara, H&M, ...)",1 Pair of Nike Running Shoes (Mid-Range),1 Pair of Men Leather Business Shoes,Apartment (1 bedroom) in City Centre,Apartment (1 bedroom) Outside of Centre,Apartment (3 bedrooms) in City Centre,Apartment (3 bedrooms) Outside of Centre,Price per Square Meter to Buy Apartment in City Centre,Price per Square Meter to Buy Apartment Outside of Centre,Average Monthly Net Salary (After Tax)
0,0,Turin,65.63,15.0,60.0,8.00,4.00,4.00,1.41,1.98,...,31.92,76.41,120.91,568.75,420.31,1107.24,765.17,3435.42,1976.00,1413.44
1,1,Milan,75.78,15.0,60.0,8.00,5.00,5.00,1.51,2.54,...,35.83,90.99,138.49,1167.00,748.08,2507.87,1514.29,8549.13,3786.26,1651.84
2,2,Chemnitz,0.00,8.5,40.0,7.75,3.00,3.75,3.00,2.08,...,24.00,68.75,88.75,385.00,332.00,850.00,675.00,3000.00,1516.67,1962.50
3,3,Dublin,77.85,15.0,62.5,8.50,5.75,6.00,3.29,1.86,...,34.70,83.30,97.38,1709.85,1434.79,3035.56,2372.22,5587.01,4026.59,2490.81
4,4,Bonn,66.01,9.5,55.0,7.50,3.50,3.50,2.61,2.06,...,35.50,72.88,100.43,784.62,586.15,1507.69,1138.46,7433.33,4933.33,2431.83
